# Creating Historical Database

In [137]:
import os
import zipfile
import numpy as np
import pandas as pd

# fetch tables from PDF
import camelot

In [11]:
datapath = './data'
unzip_path = './unzipped_data'

## unzipping files

In [7]:
zipNames = os.listdir(datapath)

In [10]:
for file in zipNames:
    with zipfile.ZipFile(f'./data/{file}', 'r') as zip_ref:
        zip_ref.extractall('./unzipped_data')

## Importing data

In [12]:
unzipNames = os.listdir(unzip_path)

In [13]:
df = pd.read_table(f'./unzipped_data/{unzipNames[0]}')

In [14]:
df.head()

,00COTAHIST.2011BOVESPA 20111229
0,012011010302ABCB4 010ABC BRASIL PN EJ ...
1,012011010396ABCB4F 020ABC BRASIL PN EJ ...
2,012011010312ABCP11 010FII ABC IMOBCI ER ...
3,012011010302AEDU3 010ANHANGUERA ON ...
4,012011010396AEDU3F 020ANHANGUERA ON ...


## Getting layout info

In [196]:
# searches tables in all pages
layout_tb = camelot.read_pdf('SeriesHistoricas_Layout.pdf', pages='all', flavor='lattice')

# fetch all tables and adjusts format
tables = []
for n in range(layout_tb.n):
    temp = layout_tb[n].df
#     temp.columns = [str(j).replace('\n','').strip() for j in temp.iloc[0].values]
    temp.columns = ['NAME', 'CONTENT','KIND AND SIZE', 'INIT', 'END']
    temp.drop(0,axis=0, inplace=True)
    tables.append(temp)

In [197]:
# concatenating tables into one big table
tb = pd.DataFrame()
header = np.NAN
for i in range(len(tables)):
    if not tables[i]['CONTENT'].str.contains('COTAHIST.AAAA').sum()>0:  # if not HEADER, add to big table
        tb = pd.concat([tb,tables[i]], axis=0)
    else:
        header=tables[i]

### removing special characters

In [198]:
tb['NAME'] = tb['NAME'].str.replace('\n','')
tb['CONTENT'] = tb['CONTENT'].str.replace('V\n','').str.replace('\n','').str.replace(' ER ',' VER ')

### treating info fields

In [199]:
tb['PREFIX'] = tb['NAME'].apply(lambda x:str(x).split(' -')[0])

In [200]:
tb

,NAME,CONTENT,KIND AND SIZE,INIT,END,PREFIX
1,TIPREG - TIPO DE REGISTRO,FIXO “01”,N(02),01,02,TIPREG
2,DATA DO PREGÃO,FORMATO “AAAAMMDD”,N(08),03,10,DATA DO PREGÃO
3,CODBDI - CÓDIGO BDI,UTILIZADO PARA CLASSIFICAR OS PAPÉIS NA EMISSÃ...,X(02),11,12,CODBDI
4,CODNEG - CÓDIGO DE NEGOCIAÇÃO DO PAPEL,,X(12),13,24,CODNEG
5,TPMERC - TIPO DE MERCADO,CÓD. DO MERCADO EM QUE O PAPEL ESTÁ CADASTRADO...,N(03),25,27,TPMERC
6,NOMRES - NOME RESUMIDO DA EMPRESA EMISSORA DO ...,,X(12),28,39,NOMRES
1,ESPECI - ESPECIFICAÇÃO DO PAPEL,VER TABELA ANEXA,X(10),40,49,ESPECI
2,PRAZOT - PRAZO EM DIAS DO MERCADO A TERMO,,X(03),50,52,PRAZOT
3,MODREF - MOEDA DE REFERÊNCIA,MOEDA USADA NA DATA DO PREGÃO,X(04),53,56,MODREF
4,PREABE - PREÇO DE ABERTURA DO PAPEL-MERCADO NO...,,(11)V99,57,69,PREABE


In [202]:
header

,NAME,CONTENT,KIND AND SIZE,INIT,END
1,TIPO DE REGISTRO,FIXO “99”,N(02),01,02
2,NOME DO ARQUIVO,FIXO “COTAHIST.AAAA”,X(13),03,15
3,CÓDIGO DA ORIGEM,FIXO “BOVESPA”,X(08),16,23
4,DATA DA GERAÇÃO DO \nARQUIVO,FORMATO “AAAAMMDD”,N(08),24,31
5,T\nOTAL DE REGISTROS,INCLUIR TAMBÉM OS \nREGISTROS HEADER E \nTRAILER.,N(11),32,42
6,RESERVA,PREENCHER COM \nBRANCOS,X(203),43,245
